In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir ~/.kaggle

# copying api file
!cp drive/MyDrive/kaggle.json ~/.kaggle/
# modding permission
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# downloading data using api command from the site
!kaggle datasets download -d ardikasatria/datasettanamanpadisumatera
# unzip the downloaded file
!unzip datasettanamanpadisumatera.zip

  0% 0.00/4.75k [00:00<?, ?B/s]
100% 4.75k/4.75k [00:00<00:00, 12.8MB/s]
Archive:  datasettanamanpadisumatera.zip
  inflating: Data_Tanaman_Padi_Sumatera_version_1.csv  


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer

In [ ]:
data = pd.read_csv('/content/Data_Tanaman_Padi_Sumatera_version_1.csv')
le = LabelEncoder()
data['Provinsi'] = le.fit_transform(data['Provinsi'])
data.head()

,Provinsi,Tahun,Produksi,Luas Panen,Curah hujan,Kelembapan,Suhu rata-rata
0,0,1993,1329536.0,323589.0,1627.0,82.00,26.06
1,0,1994,1299699.0,329041.0,1521.0,82.12,26.92
2,0,1995,1382905.0,339253.0,1476.0,82.72,26.27
3,0,1996,1419128.0,348223.0,1557.0,83.00,26.08
4,0,1997,1368074.0,337561.0,1339.0,82.46,26.31


In [ ]:
x = data[['Provinsi','Tahun','Luas Panen','Curah hujan','Kelembapan','Suhu rata-rata']]
y = data[['Produksi']]
column = x.columns

In [ ]:
x

,Provinsi,Tahun,Luas Panen,Curah hujan,Kelembapan,Suhu rata-rata
0,0,1993,323589.00,1627.0,82.00,26.06
1,0,1994,329041.00,1521.0,82.12,26.92
2,0,1995,339253.00,1476.0,82.72,26.27
3,0,1996,348223.00,1557.0,83.00,26.08
4,0,1997,337561.00,1339.0,82.46,26.31
...,...,...,...,...,...,...
219,3,2016,390799.00,2317.6,79.40,26.45
220,3,2017,396559.00,1825.1,77.04,26.36
221,3,2018,511940.93,1385.8,76.05,25.50
222,3,2019,464103.42,1706.4,78.03,27.23


In [ ]:
y

,Produksi
0,1329536.00
1,1299699.00
2,1382905.00
3,1419128.00
4,1368074.00
...,...
219,3831923.00
220,4090654.00
221,2488641.91
222,2164089.33


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 100)

svr = SVR(kernel="rbf", C=1000000.0, epsilon=0.1)
svr.fit(x_train, y_train.values.ravel())

print('R squared: {:.4f}'.format(svr.score(x_test,y_test)*100))

R squared: 82.7003


In [ ]:
from sklearn.linear_model import LinearRegression

results = []
pca_results = []

for i in range(100):
  scaler = MinMaxScaler()
  pca = PCA(n_components=1)
  svr = LinearRegression()

  pipeline = Pipeline([('scaler', scaler), ('model', svr)])
  pipeline2 = Pipeline([('scaler', scaler), ('ct', ColumnTransformer([("pca", pca, [3, 4, 5]), ("pass", "passthrough", [0, 1, 2])])), ('model', svr)])

  cv = KFold(n_splits =5, shuffle=True, random_state=i)
  R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
  results.append(R2_scores.mean())
  R2_scores2 = cross_val_score(pipeline2, x, y.values.ravel(), cv= cv)
  pca_results.append(R2_scores2.mean())
  print("")

  print(f"R2 : {R2_scores.mean()*100} | {R2_scores2.mean()*100} = {i}")


R2 : 86.89117137034856 | 86.79805973550894 = 0

R2 : 86.80802467444244 | 86.62270538053998 = 1

R2 : 86.82306069456341 | 86.92670204163637 = 2

R2 : 85.93954449537144 | 86.0442377375389 = 3

R2 : 86.3788870641966 | 86.2534369258995 = 4

R2 : 86.78868945109976 | 86.95354199568257 = 5

R2 : 85.7650738812803 | 86.22540860777794 = 6

R2 : 87.18377748419158 | 86.91859196523343 = 7

R2 : 86.42659282702596 | 86.233551800928 = 8

R2 : 86.63779501806624 | 86.64220299400142 = 9

R2 : 87.44253320692896 | 87.51084993585539 = 10

R2 : 86.59452264108909 | 86.82429204022543 = 11

R2 : 86.72482937651542 | 86.92542750674599 = 12

R2 : 86.80136361018455 | 87.11191666483768 = 13

R2 : 86.6546419262719 | 86.55041100111389 = 14

R2 : 87.36141223423779 | 87.39938596769903 = 15

R2 : 87.25954025518942 | 87.08170156885109 = 16

R2 : 87.36394415324608 | 87.26974393857562 = 17

R2 : 87.35276553932198 | 87.3254947641865 = 18

R2 : 86.0857841645698 | 86.18081221399981 = 19

R2 : 87.33743751051094 | 87.1860396344

In [ ]:
results = np.array(results)
results.mean()

pca_results = np.array(pca_results)
pca_results.mean()

print(f"{results.mean()*100} {pca_results.mean()*100}")

86.84983811067939 86.90686057723185


Base Accuracy

In [ ]:
scaler = MinMaxScaler()
svr = SVR(kernel="rbf", C=1000000.0, epsilon=0.1)

pipeline = Pipeline([('scaler', scaler), ('model', svr)])

cv = KFold(n_splits =5, shuffle=True, random_state=100)
R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
MAE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv,scoring="neg_mean_absolute_error")
MSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_mean_squared_error")
RMSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_root_mean_squared_error")

for i, (train_index, test_index) in enumerate(cv.split(x)):
  print(f"Iteration {i+1}:")
  print(f"Score: R2 {R2_scores[i]}, MAE = {MAE_scores[i]}, MSE = {MSE_scores[i]}, RMSE = {RMSE_scores[i]}")

print("")

print(f"R2 : {R2_scores.mean()*100}")
print(f"MAE : {MAE_scores.mean()}")
print(f"MSE : {MSE_scores.mean()}")
print(f"RMSE : {RMSE_scores.mean()}")

Iteration 1:
Score: R2 0.8769304264172076, MAE = -182787.53665392604, MSE = -178703699834.11932, RMSE = -422733.6038619586
Iteration 2:
Score: R2 0.7852931992957992, MAE = -200296.60940093323, MSE = -217249869564.4304, RMSE = -466100.70753478847
Iteration 3:
Score: R2 0.9093852923085397, MAE = -160638.44242384567, MSE = -135506514223.0426, RMSE = -368112.09464379546
Iteration 4:
Score: R2 0.8888874456128959, MAE = -209406.52445712942, MSE = -170031652540.03116, RMSE = -412348.9451181258
Iteration 5:
Score: R2 0.8815272937381009, MAE = -167119.70581354052, MSE = -112843424242.41022, RMSE = -335921.7531545259

R2 : 86.84047314745087
MAE : -184049.76374987495
MSE : -162867032080.80673
RMSE : -401043.42086263886


Weather Variables

In [ ]:
scaler = MinMaxScaler()
svr = SVR(kernel="rbf", C=1000000.0, epsilon=0.1)
pca = PCA(n_components=1)

pipeline = Pipeline([('scaler', scaler), ('ct', ColumnTransformer([("pca", pca, [3, 4, 5]), ("pass", "passthrough", [0, 1, 2])])), ('model', svr)])

cv = KFold(n_splits =5, shuffle=True, random_state=100)
R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
MAE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv,scoring="neg_mean_absolute_error")
MSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_mean_squared_error")
RMSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_root_mean_squared_error")

for i, (train_index, test_index) in enumerate(cv.split(x)):
  print(f"Iteration {i+1}:")
  print(f"Score: R2 {R2_scores[i]}, MAE = {MAE_scores[i]}, MSE = {MSE_scores[i]}, RMSE = {RMSE_scores[i]}")

print("")

print(f"R2 : {R2_scores.mean()*100}")
print(f"MAE : {MAE_scores.mean()}")
print(f"MSE : {MSE_scores.mean()}")
print(f"RMSE : {RMSE_scores.mean()}")

Iteration 1:
Score: R2 0.8867121450092385, MAE = -170047.71484886133, MSE = -164500113584.13412, RMSE = -405586.1358381646
Iteration 2:
Score: R2 0.7795200891757833, MAE = -192116.76757192553, MSE = -223091358592.44894, RMSE = -472325.47950798605
Iteration 3:
Score: R2 0.9082781656096248, MAE = -157270.87400016063, MSE = -137162127131.75551, RMSE = -370354.0564537609
Iteration 4:
Score: R2 0.8954524621682516, MAE = -175376.72406892656, MSE = -159985437510.4418, RMSE = -399981.7964738418
Iteration 5:
Score: R2 0.8728673969857382, MAE = -166203.47788897934, MSE = -121091842244.79869, RMSE = -347982.53152248706

R2 : 86.85660517897273
MAE : -172203.11167577066
MSE : -161166175812.71582
RMSE : -399245.9999592481


All Variables

In [ ]:
scaler = MinMaxScaler()
svr = SVR(kernel="rbf", C=1000000.0, epsilon=0.1)
pca = PCA(n_components=1)

pipeline = Pipeline([('scaler', scaler), ("pca", pca), ('model', svr)])

cv = KFold(n_splits=5, shuffle=True, random_state=65)
R2_scores = cross_val_score(pipeline, x, y.values.ravel(), cv= cv)
MAE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv,scoring="neg_mean_absolute_error")
MSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_mean_squared_error")
RMSE_scores = cross_val_score(pipeline, x, y.values.ravel(), cv = cv, scoring="neg_root_mean_squared_error")

for i, (train_index, test_index) in enumerate(cv.split(x)):
  print(f"Iteration {i+1}:")
  print(f"Score: R2 {R2_scores[i]}, MAE = {MAE_scores[i]}, MSE = {MSE_scores[i]}, RMSE = {RMSE_scores[i]}")

print("")

print(f"R2 : {R2_scores.mean()*100}")
print(f"MAE : {MAE_scores.mean()}")
print(f"MSE : {MSE_scores.mean()}")
print(f"RMSE : {RMSE_scores.mean()}")

Iteration 1:
Score: R2 0.4601156315578614, MAE = -570344.7839278064, MSE = -712202730766.6705, RMSE = -843921.0453393555
Iteration 2:
Score: R2 0.7295122297897891, MAE = -404975.331975241, MSE = -262906526651.01038, RMSE = -512744.1142041617
Iteration 3:
Score: R2 0.8216653048721496, MAE = -344045.03141532384, MSE = -205029147856.69952, RMSE = -452801.4441857485
Iteration 4:
Score: R2 0.4245798724503814, MAE = -681274.2882132721, MSE = -1112594868209.3408, RMSE = -1054796.1263719832
Iteration 5:
Score: R2 0.8260993288885506, MAE = -358026.6588959839, MSE = -210099984680.8963, RMSE = -458366.6487440991

R2 : 65.23944735117465
MAE : -471733.2188855254
MSE : -500566651632.9235
RMSE : -664525.8757690697
